In [1]:
import torch
from sklearn.feature_extraction.text import CountVectorizer
from transformers import BertTokenizer, BertModel
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
import requests
from bs4 import BeautifulSoup
import json
import os
from datetime import datetime

import pandas as pd

import re
import numpy as np
from collections import Counter
from konlpy.tag import Komoran
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

from sklearn.linear_model import LogisticRegression

import joblib

## 1. '오늘' update 된 공지만 크롤링 해오기

In [3]:
# User-Agent 헤더 설정
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

def fetch_notices():
    notices = []
    offset = 0
    today = datetime.today().strftime('%Y-%m-%d')
    more_pages = True

    while more_pages:
        # 공지사항 목록 페이지 URL (페이지 넘버링 적용)
        url = f"https://ewha.ac.kr/ewha/news/notice.do?mode=list&&articleLimit=10&article.offset={offset}"

        # 페이지 가져오기
        response = requests.get(url, headers=headers)
        html = response.text

        # BeautifulSoup을 사용하여 HTML 파싱
        soup = BeautifulSoup(html, 'html.parser')

        # 공지사항 행 추출
        page_notices = soup.find_all('tr')

        # 고정된 공지사항은 가져오지 말고, 일반 공지사항만 가져옴
        for notice in page_notices:
            # 고정된 공지는 <tr class="b top box">가 있음
            if 'b-top-box' in notice.get('class', []):
                continue

            # 일반 공지는 클래스가 없는 <tr> 태그를 사용
            num_box = notice.find('td', class_='b-num-box')

            if num_box:  # 일반 공지일 때만 처리
                title = notice.find('td', class_='b-td-left').find('div', class_='b-title-box').find('a').get_text(strip=True)
                date = notice.find('span', class_='b-date').get_text(strip=True)
                date = date.replace('.', '-')

                # 만약 오늘의 날짜가 아니면 더 이상 크롤링하지 않음
                if date != today:
                    more_pages = False
                    break  # 현재 페이지의 나머지도 처리하지 않음

                # 오늘 날짜의 공지만 저장
                notices.append({'title': title, 'date': date})

        if more_pages:
            offset += 10  # 오늘 날짜인 공지가 있을 경우에만 다음 페이지로 이동

    return notices

# 공지사항 가져오기
all_notices = fetch_notices()

# 가져온 공지사항 출력
for notice in all_notices:
    print(f"제목: {notice['title']}, 등록일: {notice['date']}")


제목: [해저드리터러시융합교육 연구소] 계약직원 채용 공고, 등록일: 2024-09-26
제목: [입학] 2025학년도 전기 공연예술대학원(야간) 신입생 모집(10/10~10/24), 등록일: 2024-09-26
제목: [채용]이화여자대학교 학교폭력예방연구소 연구원 모집 공고, 등록일: 2024-09-26
제목: [건축팀] 공사로 인한 통행제한 안내 (9/29 일 ~ 9/30 월), 등록일: 2024-09-26
제목: 디자인씽킹 워크숍 : 창의적 해결법을 찾아가는 과정별 방법론 특강 (휴학생 가능), 등록일: 2024-09-26
제목: [채용] 사범대학 과학교육과 학과사무실 계약직원 채용 공고, 등록일: 2024-09-26
제목: [인재] 2024년도 5급(행정) 및 외교관후보자선발시험 3차 대비 프로그램 안내(2차 합격자 대상), 등록일: 2024-09-26
제목: [조교모집] 법전원 졸업시험 감독조교 모집 (~10/7), 등록일: 2024-09-26


In [4]:
# 'date' 필드를 제거한 새로운 리스트 생성
all_notices= [notice['title'] for notice in all_notices]

print(all_notices)

['[해저드리터러시융합교육 연구소] 계약직원 채용 공고', '[입학] 2025학년도 전기 공연예술대학원(야간) 신입생 모집(10/10~10/24)', '[채용]이화여자대학교 학교폭력예방연구소 연구원 모집 공고', '[건축팀] 공사로 인한 통행제한 안내 (9/29 일 ~ 9/30 월)', '디자인씽킹 워크숍 : 창의적 해결법을 찾아가는 과정별 방법론 특강 (휴학생 가능)', '[채용] 사범대학 과학교육과 학과사무실 계약직원 채용 공고', '[인재] 2024년도 5급(행정) 및 외교관후보자선발시험 3차 대비 프로그램 안내(2차 합격자 대상)', '[조교모집] 법전원 졸업시험 감독조교 모집 (~10/7)']


In [5]:
# 결과를 pandas 데이터프레임으로 변환
df = pd.DataFrame(all_notices, columns=['title'])
df.head()

,title
0,[해저드리터러시융합교육 연구소] 계약직원 채용 공고
1,[입학] 2025학년도 전기 공연예술대학원(야간) 신입생 모집(10/10~10/24)
2,[채용]이화여자대학교 학교폭력예방연구소 연구원 모집 공고
3,[건축팀] 공사로 인한 통행제한 안내 (9/29 일 ~ 9/30 월)
4,디자인씽킹 워크숍 : 창의적 해결법을 찾아가는 과정별 방법론 특강 (휴학생 가능)


In [6]:
df1 = df.copy()

## 2. 전처리 실행

In [7]:
#영문자와 한글만 title에 남기기

df1['title'] = df1['title'].str.replace('[^a-z|A-Z|ㄱ-ㅎ|가-힣|]', ' ', regex = True)

# lambda 함수를 사용해서 text의 탐지된 x를 lowercase로 바꿔준다.
def lowercase(text):
    return re.sub(r'[a-zA-Z]', lambda x: x.group().lower(), text)

df1['title'] = df1['title'].apply(lowercase)

# 영어와 한국어가 섞여 있으므로 구분할 수 있도록
eng = re.compile(r'[a-zA-Z]')
    
#불용어 사전
stop_words = ['의','가','이','은','들','는','걍','과','들','과','으로','도','을',
              '를','으로','자','에','와','한','하다','에서','에게', '및', '연도', '년',
              '년도', '학기', 'ㄴ' '학년도', '회', '상반기', '하반기', '년대', '학년',
              '오후', '오전', '오늘', '내일', '회차', '개월', '주년', '종료', 'th', 'st', 'nd', "'s", '접수', '기한', '연장',
              '월', '어', '다', '까지', '제', '등', '등등', '몇', '면', '각', '각각', '마감','공통', '만료',
              '여', '대', '백양']

# 형태소 분석 및 불용어 제거 함수

def preprocess(text):
    komoran = Komoran(userdic = '/content/drive/MyDrive/유런 24 여름 방학 프로젝트/eda&전처리/user_dictionary.txt')
    lemmatizer = WordNetLemmatizer()

    token_list = []

    for sentence in text :
       sentence = sentence.strip()
       if not sentence:
           token_list.append('')
           continue

       words = sentence.split()
       sentense_list = []

       for word in words :
          if eng.match(word) :
              tokens_word = word_tokenize(word)
              tokens_pos = pos_tag(tokens_word)
              for w, pos in tokens_pos:
                  if pos.startswith('N'):
                      lemma = lemmatizer.lemmatize(w, pos='n')
                  elif pos.startswith('V'):
                      lemma = lemmatizer.lemmatize(w, pos='v')
                  elif pos.startswith('J'):
                      lemma = lemmatizer.lemmatize(w, pos='a')
                  elif pos.startswith('R'):
                      lemma = lemmatizer.lemmatize(w, pos='r')
                  else :
                      continue
                  sentense_list.append(lemma)

          else :
              nouns = komoran.nouns(word)
              sentense_list.extend(nouns)
       token = [t for t in sentense_list if t not in stop_words]
       token_list.append(' '.join(token))
    return token_list

df1['processed_title'] = preprocess(df1['title'])

# 한 글자 단어를 제거하되 "팀"과 "랩" 단어는 유지하는 함수

def remove_single_characters(text):
    words = text.split()
    filtered_words = [word for word in words if len(word) > 1 or word in ['팀', '랩']]
    return ' '.join(filtered_words)

df1['processed_title'] = df1['processed_title'].apply(remove_single_characters)



In [8]:
df

,title
0,[해저드리터러시융합교육 연구소] 계약직원 채용 공고
1,[입학] 2025학년도 전기 공연예술대학원(야간) 신입생 모집(10/10~10/24)
2,[채용]이화여자대학교 학교폭력예방연구소 연구원 모집 공고
3,[건축팀] 공사로 인한 통행제한 안내 (9/29 일 ~ 9/30 월)
4,디자인씽킹 워크숍 : 창의적 해결법을 찾아가는 과정별 방법론 특강 (휴학생 가능)
5,[채용] 사범대학 과학교육과 학과사무실 계약직원 채용 공고
6,[인재] 2024년도 5급(행정) 및 외교관후보자선발시험 3차 대비 프로그램 안내(...
7,[조교모집] 법전원 졸업시험 감독조교 모집 (~10/7)


In [9]:
df1

,title,processed_title
0,해저드리터러시융합교육 연구소 계약직원 채용 공고,해저드 리터 러시 융합 교육 연구소 계약 직원 채용 공고
1,입학 학년도 전기 공연예술대학원 야간 신입생 모집,입학 전기 공연 예술 대학원 야간 신입생 모집
2,채용 이화여자대학교 학교폭력예방연구소 연구원 모집 공고,채용 이화여자대학교 학교 폭력 예방 연구소 연구원 모집 공고
3,건축팀 공사로 인한 통행제한 안내 일 월,건축 팀 공사 통행 제한 안내
4,디자인씽킹 워크숍 창의적 해결법을 찾아가는 과정별 방법론 특강 휴학생 가능,워크숍 창의 해결 과정 방법론 특강 휴학
5,채용 사범대학 과학교육과 학과사무실 계약직원 채용 공고,채용 사범대학 과학 교육 학과 사무실 계약 직원 채용 공고
6,인재 년도 급 행정 및 외교관후보자선발시험 차 대비 프로그램 안내 ...,인재 행정 외교관 후보자 선발 시험 대비 프로그램 안내 합격자 대상
7,조교모집 법전원 졸업시험 감독조교 모집,조교 모집 전원 졸업 시험 감독 조교 모집


## 3. NLP 모델로 커리어 관련 공지 분류하기

In [10]:
# N-그램 벡터화 (1-그램, 2-그램)
vectorizer = joblib.load("C:/Users/Flex/Documents/Euron 6th/ngram_vectorizer.pkl")
X_ngrams = vectorizer.transform(df1['processed_title']).toarray()

print(vectorizer.get_feature_names_out())
print(X_ngrams)

['abeek' 'abeek 연구원' 'abeek 자체' ... '희망 청년' '희망자' '희망자 모집']
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


C:\Users\Flex\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator CountVectorizer from version 1.5.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [11]:
# KoBERT 모델과 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertModel.from_pretrained('monologg/kobert')

# GPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
C:\Users\Flex\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(8002, 768, padding_idx=1)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)

In [12]:
# 문장 임베딩 추출
def get_sentence_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True, max_length=128)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # 데이터를 GPU로 이동
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()  # 결과를 CPU로 다시 이동

In [13]:
# 모든 문장에 대해 임베딩 추출
embeddings = [get_sentence_embedding(sentence) for sentence in df1['processed_title']]
X_embeddings = pd.DataFrame(embeddings)

In [14]:
X_combined = np.hstack((X_ngrams, X_embeddings))

In [15]:
# ML model load
LR = joblib.load("C:/Users/Flex/Documents/Euron 6th/logistic_regression_model.pkl")

C:\Users\Flex\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.5.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [16]:
y_pred = LR.predict(X_combined)

y_pred

array([1, 0, 1, 0, 1, 1, 1, 0], dtype=int64)

In [17]:
df1['label'] = y_pred

df1

,title,processed_title,label
0,해저드리터러시융합교육 연구소 계약직원 채용 공고,해저드 리터 러시 융합 교육 연구소 계약 직원 채용 공고,1
1,입학 학년도 전기 공연예술대학원 야간 신입생 모집,입학 전기 공연 예술 대학원 야간 신입생 모집,0
2,채용 이화여자대학교 학교폭력예방연구소 연구원 모집 공고,채용 이화여자대학교 학교 폭력 예방 연구소 연구원 모집 공고,1
3,건축팀 공사로 인한 통행제한 안내 일 월,건축 팀 공사 통행 제한 안내,0
4,디자인씽킹 워크숍 창의적 해결법을 찾아가는 과정별 방법론 특강 휴학생 가능,워크숍 창의 해결 과정 방법론 특강 휴학,1
5,채용 사범대학 과학교육과 학과사무실 계약직원 채용 공고,채용 사범대학 과학 교육 학과 사무실 계약 직원 채용 공고,1
6,인재 년도 급 행정 및 외교관후보자선발시험 차 대비 프로그램 안내 ...,인재 행정 외교관 후보자 선발 시험 대비 프로그램 안내 합격자 대상,1
7,조교모집 법전원 졸업시험 감독조교 모집,조교 모집 전원 졸업 시험 감독 조교 모집,0


In [34]:
# 'label'이 1인 행들의 인덱스 추출
indices = df1[df1['label'] == 1].index

# 해당 인덱스의 'title' 열 값 가져오기
career_notices = df.loc[indices, 'title']

def organize_title(notices):
    if notices is None or len(notices) == 0:  
        return "오늘은 커리어 관련 공지사항이 없습니다."
    
    # 공지사항이 있을 때 문자열 생성
    organized_list = '\n'.join([f"🔸 {title}" for title in notices])
    return organized_list

## 4. 이메일로 알림 보내기

In [46]:
import os

# Jupyter Notebook에서 환경 변수 설정
os.environ['EMAIL_PASSWORD'] = 'ryot gxkt zhyb hbwn'

In [48]:
import os
import smtplib
from email.mime.text import MIMEText

def send_email(subject, body, to_email):
    from_email = "eunbin3660@gmail.com"
    password = os.getenv('EMAIL_PASSWORD')  # 앱 비밀번호를 환경 변수로 처리

    # 이메일 메시지 생성
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] = from_email
    msg['To'] = to_email

    try:
        # Gmail SMTP 서버에 연결
        server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        server.login(from_email, password)
        server.sendmail(from_email, to_email, msg.as_string())
        server.quit()
        print("이메일 전송 성공!")
    except smtplib.SMTPAuthenticationError as e:
        print(f"SMTP 인증 오류: {e}")
    except Exception as e:
        print(f"이메일 전송 중 오류 발생: {e}")

# 커리어 관련 공지가 있다면 이메일로 보내기
if not career_notices.empty:
    subject = "오늘의 커리어 관련 공지사항"
    body = organize_title(career_notices)
    send_email(subject, body, "eunbin3660@gmail.com")


1
2
3
이메일 전송 성공!


In [45]:
print(os.getenv('EMAIL_PASSWORD'))

None
